In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("../")
import rsc.Resources_New as rsc
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Here i the csv that ill be using.
df_movies = pd.read_csv("../Data/imdb_top_1000.csv")

#I import the list that i scrapped with seleneum thats saved in a pickle so i dont lose the info when the kernel shuts down.
with open("../Data/libros_scrap.pkl", "rb") as alias:
    libro2 =pickle.load(alias)

In [3]:
#Seleneum configurations.

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window


In [4]:
#This loop runs my seleneum with its function.
libro = []
resultado = []
for i in df_movies["Series_Title"]:
    rsc.busqueda(i,resultado)
    print(resultado)
    libro.append(resultado)
    resultado = []

['Stephen King', '4.47', '2,457 ratings', 'The Shawshank Redemption: Different Seasons']
['Mario Puzo', '4.38', '390,953 ratings', 'The Godfather']
['Frank Miller', '4.26', '179,873 ratings', 'Batman: The Dark Knight Returns']
['Elizabeth Tubbs', '5.00', '1 rating', 'The Godfather Part II: Screenplay']
['Reginald Rose', '4.07', '29,314 ratings', 'Twelve Angry Men']
['Jude Fisher', '4.62', '22,180 ratings', 'The Lord of the Rings: The Return of the King: Visual Companion']
['Quentin Tarantino', '4.20', '6,016 ratings', 'Pulp Fiction: A Quentin Tarantino Screenplay']
['Thomas Keneally', '4.34', '156,003 ratings', "Schindler's List"]
['Bianca Scardoni', '3.99', '11,775 ratings', 'Inception']


In [ ]:
#Here i run all of my cleaning functions.
scrap_limpio = rsc.listas_libros(libro2)
lista_votos_limpio = [rsc.limpieza_votos(voto) for voto in scrap_limpio[2]]
nota_promedio_limpio = [rsc.promedio(rati) for rati in scrap_limpio[1]]
Autor_limpio = [rsc.Autor(auto) for auto in scrap_limpio[0]]
fuzzy_titulo = rsc.semejanza(scrap_limpio[3],list(df_movies["Series_Title"]))
df_movies["Gross"] = df_movies["Gross"].str.replace(',','')

In [ ]:
#I create columns in my movie DataFrame so i can then match movies with books
df_movies["Existe_libro"] = fuzzy_titulo
df_movies["Autor_libro"] = Autor_limpio
df_movies["Nota_promedio_libro"] = nota_promedio_limpio
df_movies["Votos_libro"] = lista_votos_limpio
df_movies['Nota_promedio_libro'] = df_movies['Nota_promedio_libro'].apply(lambda x: x*2)

In [ ]:
#I create a new DataFrame with only the rows that match that there is an existing book.
df_movies_limpio = df_movies[df_movies["Existe_libro"] == True]

#Here i copy all of the columns that i will use.
df_libros = df_movies_limpio[["Autor_libro","Nota_promedio_libro","Votos_libro"]].copy()

#And here i eliminate all of the columns that i won't be using anymore.

df_movies_limpio.drop("Existe_libro",inplace=True,axis=1),




In [ ]:
#And finally i create the new CSV with all of the data that ill need for this project.
df_movies_limpio.to_csv(f"../Data/df_movies_limpio.csv")
